<a href="https://colab.research.google.com/github/SunSlick2/booktrade/blob/main/hmmm_no_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No output in the console/IDLE *and* no `extracted_date_easyocr.txt` file being created suggests a more fundamental problem. This points to the Python script either:

1.  **Crashing immediately on launch** (before any `sys.stdout.write` or file operations can happen).
2.  **Not actually being run at all** when you think it is.

Let's systematically troubleshoot this.

### Troubleshooting Steps (The "No Output, No File" Edition)

#### Step 1: Verify the Python Script's Execution Path

  * **How are you running the script?**

      * Are you running it directly from an Anaconda Prompt/terminal?
      * Are you double-clicking the `.py` file? (This often just flashes a window and closes if there's an error.)
      * Are you running it from an IDE (like VS Code, PyCharm, or Spyder)?
      * Are you using IDLE's "Run Module" (F5)?
      * Are you calling it from VBA?

  * **Crucial Test: Run from a Controlled Environment:**

    1.  **Open a fresh Anaconda Prompt** (or a regular Command Prompt if you don't use Anaconda for this project).
    2.  **Activate your environment** (if using one): `conda activate your_easyocr_env`
    3.  **Navigate to the exact directory** where your `DateOCR_TesseractFailed.py` (or whatever you've named the latest version) is located:
        `cd C:\Users\abc\OneDrive\Documents\Tools\PythonScripts`
    4.  **Execute the script directly:**
        `python DateOCR_TesseractFailed.py` (replace with your actual filename)

    **What to look for:**

      * **Does *anything* print to this command prompt window?** Even error messages will appear here.
      * **Does the `extracted_date_easyocr.txt` file appear** in `C:\Users\abc\OneDrive\Documents\Tools\PythonScripts`?
      * **Does `captured_date_region_debug.png` appear** in the same directory?

    **If you still get no output and no files when running like this, it points to a very early crash.**

#### Step 2: Debugging Immediate Crashes

If the script crashes instantly, it's usually due to:

  * **Syntax Errors:** A typo that prevents Python from even parsing the file.
  * **Missing Core Imports:** A fundamental library like `mss` or `PIL` isn't installed or is corrupted.
  * **Incorrect `os.environ` Placement:** Though less likely to cause a *complete* crash, if the `os.environ` line is misplaced or `os` isn't imported first, it could be an issue.

**Let's add aggressive logging/tracing to pinpoint the failure point:**

In [ ]:
import sys
# Start of script trace
sys.stdout.write("--- Script started ---\n")
sys.stdout.flush()

import os
# Set OpenMP environment variable to suppress warnings (must be at the very top)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
sys.stdout.write("Environment variable KMP_DUPLICATE_LIB_OK set.\n")
sys.stdout.flush()

import mss
import mss.tools
from PIL import Image
import easyocr
import re

sys.stdout.write("All core modules imported.\n")
sys.stdout.flush()

# --- Configuration ---
# Define the snip coordinates as (top_left_x, top_left_y) and (bottom_right_x, bottom_right_y)
top_left_x = 317
top_left_y = 419
bottom_right_x = 428
bottom_right_y = 456

# Calculate width and height from the provided coordinates
capture_region = {
    "left": top_left_x,
    "top": top_left_y,
    "width": bottom_right_x - top_left_x,
    "height": bottom_right_y - top_left_y
}

# Define the path for the output file
output_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "extracted_date_easyocr.txt")
sys.stdout.write(f"Output file path set to: {output_file_path}\n")
sys.stdout.flush()


# Define the directory where EasyOCR models are stored locally (e.g., after manual download)
easyocr_model_dir = os.path.join(os.path.expanduser('~'), '.EasyOCR', 'model')
sys.stdout.write(f"EasyOCR model directory set to: {easyocr_model_dir}\n")
sys.stdout.flush()


# Initialize EasyOCR reader once globally for efficiency
try:
    sys.stdout.write("Attempting to initialize EasyOCR reader...\n")
    sys.stdout.flush()
    reader = easyocr.Reader(
        ['en'], # Languages to use for OCR. 'en' for English.
        model_storage_directory=easyocr_model_dir,
        download_enabled=False # Crucial: tells EasyOCR NOT to try downloading from the internet
    )
    sys.stdout.write("✅ EasyOCR reader initialized successfully from local models.\n")
    sys.stdout.flush()
except Exception as e:
    sys.stderr.write(f"❌ Error initializing EasyOCR reader. Please ensure models are in '{easyocr_model_dir}' and are correct: {e}\n")
    sys.stderr.flush()
    sys.exit(1) # Exit if reader cannot be initialized


def capture_screen_region(region):
    """Captures a specific region of the screen."""
    try:
        sys.stdout.write("Capturing screen region...\n")
        sys.stdout.flush()
        with mss.mss() as sct:
            sct_img = sct.grab(region)
            # Convert to PIL Image for processing
            img = Image.frombytes("RGB", sct_img.size, sct_img.rgb)
            return img
    except Exception as e:
        sys.stderr.write(f"❌ Error capturing screen: {e}\n")
        sys.stderr.flush()
        return None

def ocr_image_for_date(image):
    """Performs OCR on an image and tries to find a date using EasyOCR."""
    if image is None:
        return None

    try:
        sys.stdout.write("Performing OCR with EasyOCR...\n")
        sys.stdout.flush()
        results = reader.readtext(image)

        full_ocr_text = ""
        for (bbox, text, prob) in results:
            full_ocr_text += text + " "

        sys.stdout.write(f"DEBUG: EasyOCR Raw Text (concatenated): {full_ocr_text.strip()}\n")
        sys.stdout.flush()

        date_pattern = r'\b(\d{1,2})[/\-.](0[1-9]|1[0-2])[/\-.]((?:19|20)\d{2})\b'

        for (bbox, text, prob) in results:
            match = re.search(date_pattern, text)
            if match:
                day = match.group(1).zfill(2)
                month = match.group(2)
                year = match.group(3)
                return f"{day}/{month}/{year}"

        match = re.search(date_pattern, full_ocr_text)
        if match:
            day = match.group(1).zfill(2)
            month = match.group(2)
            year = match.group(3)
            return f"{day}/{month}/{year}"

        return None

    except Exception as e:
        sys.stderr.write(f"❌ Error during EasyOCR processing or date parsing: {e}\n")
        sys.stderr.flush()
        return None

def write_result_to_file(value, file_path):
    """Writes the extracted value to a specified file."""
    try:
        sys.stdout.write(f"Writing result to file: {file_path}\n")
        sys.stdout.flush()
        with open(file_path, "w") as f:
            f.write(value if value is not None else "")
    except Exception as e:
        sys.stderr.write(f"❌ Error writing to file {file_path}: {e}\n")
        sys.stderr.flush()

def main():
    sys.stdout.write("Entering main function...\n")
    sys.stdout.flush()
    extracted_date = None
    try:
        screenshot = capture_screen_region(capture_region)
        if screenshot:
            screenshot_debug_path = "captured_date_region_debug.png"
            screenshot.save(screenshot_debug_path)
            sys.stdout.write(f"Captured region saved to {screenshot_debug_path}\n")
            sys.stdout.flush()

            extracted_date = ocr_image_for_date(screenshot)

    except Exception as e:
        sys.stderr.write(f"❌ An unexpected error occurred in main: {e}\n")
        sys.stderr.flush()
    finally:
        sys.stdout.write("Exiting main function (finally block)...\n")
        sys.stdout.flush()
        write_result_to_file(extracted_date, output_file_path)
        if extracted_date:
            sys.stdout.write(f"✔️ Successfully wrote '{extracted_date}' to {output_file_path}\n")
        else:
            sys.stdout.write(f"⚠️ No date found or error occurred. Wrote empty string to {output_file_path}\n")
        sys.stdout.flush()


if __name__ == "__main__":
    main()
    sys.stdout.write("--- Script finished ---\n")
    sys.stdout.flush()

**Run this extensively logged version from an Anaconda Prompt/terminal as described in Step 1.**

The goal is to see *which* `sys.stdout.write()` line is the last one to appear before the script stops. This will tell us the exact point of failure.

#### Step 3: Check Python Environment (again)

If even the very first `sys.stdout.write("--- Script started ---\n")` doesn't appear, it suggests:

  * **You're running the wrong file.** Double-check the filename you're typing in the prompt matches the file you're editing.
  * **Your Python installation itself is broken or the environment isn't activated.**
      * In the Anaconda Prompt, type `python --version`. Does it show a valid Python version (e.g., `Python 3.9.12`)?
      * Try a very simple script:
        `echo print("Hello World!") > test.py`
        `python test.py`
        Does "Hello World\!" appear? If not, your Python setup is fundamentally broken.

Let me know what you observe when running the script directly from the Anaconda Prompt with the added debug prints. The specific last message you see will be crucial.